# Laboratorio 4 — Modelos con Datos Geoespaciales

**Fecha:** 2025-08-14
**Integrantes:** Emilio Reyes, Silvia Illescas, Michelle Mejia

Este notebook sigue paso a paso las instrucciones del laboratorio para analizar cianobacteria en los lagos **Atitlán** y **Amatitlán** usando **Sentinel‑2**.


## Contenido
1. Preparación del entorno
2. Conexión a openEO (CDSE)
3. Definición de AOIs y rango temporal (≥ 6 meses)
4. Descarga/Exportación de índices (NDCI, NDVI, NDWI) como GeoTIFF
5. Conversión a NumPy y series de tiempo por lago
6. Mapas espaciales y comparación entre fechas
7. Correlación NDCI ↔ NDVI/NDWI
8. Modelos: serie temporal, clasificación y modelo híbrido
9. Visualización de proyecciones en mapa

**Nota:** El código de descarga vía openEO se ejecuta conectado a internet y con autenticación.


## 1) Preparación del entorno

In [2]:

!pip install openeo rasterio rioxarray xarray numpy pandas matplotlib folium shapely geopandas pyproj statsmodels scikit-learn

import os, glob, math, json
from datetime import date
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Reglas para gráficos
plt.rcParams['figure.figsize'] = (8,4)
plt.rcParams['axes.grid'] = True
print('Entorno listo.')


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Entorno listo.


## 2) Conexión a openEO (CDSE)
Se conecta al backend de *Copernicus Data Space Ecosystem*. Puedes autenticar con navegador (OIDC) o con credenciales de cliente.
**Ejecuta esta celda solo cuando estés listo para descargar/ procesar datos.**

In [3]:
# Conexión a openEO (ejecuta cuando tengas internet y credenciales)
try:
    import openeo
    con = openeo.connect("https://openeo.dataspace.copernicus.eu")
    # Opción A: autenticación interactiva (abre navegador)
    con = con.authenticate_oidc()
    #
    # Opción B: client credentials (si ya registraste un OAuth client)
    # con = con.authenticate_oidc_client_credentials(client_id="TU_CLIENT_ID", client_secret="TU_CLIENT_SECRET")
    print("Conexión inicializada. Autentica con uno de los métodos comentados.")
except Exception as e:
    print("openeo no disponible en este entorno o falta internet/credenciales:", e)

Authenticated using refresh token.
Conexión inicializada. Autentica con uno de los métodos comentados.


## 3) AOIs y rango temporal (≥ 6 meses)
Usamos los *bounding boxes* provistos en el enunciado. Ajusta las fechas si lo deseas.

In [4]:
# Bounding boxes del laboratorio
atitlan_bbox = {"west": -91.326256, "east": -91.07151, "south": 14.5948, "north": 14.750979}
amatitlan_bbox = {"west": -90.638065, "east": -90.512924, "south": 14.412347, "north": 14.493799}

# Ventana temporal >= 6 meses
start_date = "2025-01-01"
end_date   = "2025-07-31"
collection = "SENTINEL2_L2A"  # Surface Reflectance recomendado
print("AOIs y fechas definidos.")

AOIs y fechas definidos.


## 4) Exportar índices (NDCI, NDVI, NDWI) con openEO a GeoTIFF
Creamos un cubo, filtramos nubosidad y enmascaramos usando SCL. Luego calculamos índices y guardamos como GeoTIFF.
**Nota:** Esta sección requiere conexión real a openEO/CDSE.

Esto solo aplica para atitlan, ya que amatitlan son otras fechas las que no tienen nubosidad....

In [11]:
# === Paso 4 (definitivo): un job por fecha, con retry y cálculo local de índices ===
import os, time, glob, datetime as dt
import numpy as np
import rasterio

# Fechas suministradas (deduplicadas y ordenadas)
fechas_filtradas = sorted(set([
    "2025-02-07","2025-02-10","2025-02-25","2025-02-27",
    "2025-03-02","2025-03-04","2025-03-07","2025-03-09",
    "2025-03-12","2025-03-14","2025-03-19","2025-03-22",
    "2025-03-24","2025-03-26","2025-04-03","2025-04-11",
    "2025-04-13","2025-04-15","2025-04-16","2025-04-18",
    "2025-04-28","2025-05-03","2025-05-13","2025-05-28",
    "2025-07-10","2025-07-17","2025-07-20","2025-07-24","2025-08-01"
]))

def _next_day(d):  # 'YYYY-MM-DD' -> 'YYYY-MM-DD' + 1
    return (dt.datetime.fromisoformat(d) + dt.timedelta(days=1)).date().isoformat()

def _write_singleband_tif(ref_src, array, out_path, nodata=np.nan):
    profile = ref_src.profile.copy()
    profile.update(count=1, dtype="float32", nodata=nodata)
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(array.astype("float32"), 1)

def _calc_indices_and_save(src_path, out_base, date_str):
    with rasterio.open(src_path) as src:
        if src.count < 5:
            print(f"Saltando {src_path}: se esperaban 5 bandas (B03,B04,B05,B08,SCL).")
            return
        B03 = src.read(1).astype("float32")
        B04 = src.read(2).astype("float32")
        B05 = src.read(3).astype("float32")
        B08 = src.read(4).astype("float32")
        SCL = src.read(5)

        # Máscara: agua y sin nubes/sombras
        is_water = (SCL == 6)
        is_cloud = ((SCL == 3) | (SCL == 8) | (SCL == 9) | (SCL == 10) | (SCL == 11))
        mask = is_water & (~is_cloud)

        # Escala a reflectancia
        scale = 0.0001
        B03s, B04s, B05s, B08s = B03*scale, B04*scale, B05*scale, B08*scale
        eps = 1e-6

        ndci = (B05s - B04s) / (B05s + B04s + eps)
        ndvi = (B08s - B04s) / (B08s + B04s + eps)
        ndwi = (B03s - B08s) / (B03s + B08s + eps)

        ndci[~mask] = np.nan
        ndvi[~mask] = np.nan
        ndwi[~mask] = np.nan

        for name, arr in [("ndci", ndci), ("ndvi", ndvi), ("ndwi", ndwi)]:
            out_dir = os.path.join(out_base, name)
            os.makedirs(out_dir, exist_ok=True)
            _write_singleband_tif(src, arr, os.path.join(out_dir, f"{name}_{date_str}.tif"))

def _download_one_day(con, bbox, collection_id, date_str, raw_dir, job_title, max_retries=4):
    start, end = date_str, _next_day(date_str)
    cube = con.load_collection(
        collection_id,
        spatial_extent=bbox,
        temporal_extent=[start, end],
        bands=["B03","B04","B05","B08","SCL"],
    )
    job = cube.save_result(format="GTIFF")
    backoff = 2.0
    for attempt in range(1, max_retries+1):
        try:
            batch = con.create_job(job, title=f"{job_title}_{date_str}")
            res = batch.start_and_wait()
            os.makedirs(raw_dir, exist_ok=True)
            res.download_results(raw_dir)
            print(f"✅ Descargado {job_title} {date_str}")
            return True
        except Exception as e:
            msg = str(e)
            if "Too Many Requests" in msg or "429" in msg:
                print(f"⚠️ 429 en {date_str}. Reintentando en {backoff:.1f}s (intento {attempt}/{max_retries})...")
                time.sleep(backoff)
                backoff *= 1.8
            else:
                print(f"❌ Falló {date_str}: {e}")
                return False
    print(f"❌ Agotados reintentos para {date_str}")
    return False

def pipeline_per_area(con, bbox, area_name, out_base, fechas):
    raw_dir = os.path.join(out_base, "raw")
    os.makedirs(raw_dir, exist_ok=True)
    ok = 0
    for d in fechas:
        if _download_one_day(con, bbox, collection, d, raw_dir, f"{area_name}_RAW"):
            # hallar el tif recién bajado (multibanda)
            tifs = sorted(glob.glob(os.path.join(raw_dir, "**", "*.tif"), recursive=True), key=os.path.getmtime)
            if not tifs:
                print(f"⚠️ No se encontró el GeoTIFF para {d}")
                continue
            latest = tifs[-1]
            _calc_indices_and_save(latest, out_base, d)
            ok += 1
    print(f"✔️ {area_name}: {ok}/{len(fechas)} fechas procesadas")

# Ejecuta por lago (uno a la vez, para evitar 429)
pipeline_per_area(con, atitlan_bbox,   "Atitlan",   "out_atitlan",   fechas_filtradas)
# Cuando termine Atitlán, ejecuta Amatitlán:
#pipeline_per_area(con, amatitlan_bbox, "Amatitlan", "out_amatitlan", fechas_filtradas)


0:00:00 Job 'j-2508142107054045a39a1cff211856c2': send 'start'
0:00:13 Job 'j-2508142107054045a39a1cff211856c2': created (progress 0%)
0:00:18 Job 'j-2508142107054045a39a1cff211856c2': created (progress 0%)
0:00:25 Job 'j-2508142107054045a39a1cff211856c2': created (progress 0%)
0:00:33 Job 'j-2508142107054045a39a1cff211856c2': created (progress 0%)
0:00:43 Job 'j-2508142107054045a39a1cff211856c2': created (progress 0%)
0:00:55 Job 'j-2508142107054045a39a1cff211856c2': created (progress 0%)
0:01:11 Job 'j-2508142107054045a39a1cff211856c2': running (progress N/A)
0:01:30 Job 'j-2508142107054045a39a1cff211856c2': running (progress N/A)
0:01:54 Job 'j-2508142107054045a39a1cff211856c2': running (progress N/A)
0:02:24 Job 'j-2508142107054045a39a1cff211856c2': finished (progress 100%)


C:\Users\usuario\AppData\Local\Temp\ipykernel_18132\3722479603.py:75: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  res.download_results(raw_dir)
C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openeo\rest\job.py:199: UserDeprecationWarning: Call to deprecated method get_result. (Use `BatchJob.get_results` instead.) -- Deprecated since version 0.4.10.
  return self.get_result().download_files(target)
C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openeo\rest\job.py:203: UserDeprecationWarning: Call to deprecated class _Result. (Use `JobResults` instead) -- Deprecated since version 0.4.10.
  return _Result(self)


✅ Descargado Atitlan_RAW 2025-02-07
0:00:00 Job 'j-2508142110314508a3d071f89b321023': send 'start'
0:00:13 Job 'j-2508142110314508a3d071f89b321023': created (progress 0%)
0:00:18 Job 'j-2508142110314508a3d071f89b321023': created (progress 0%)
0:00:25 Job 'j-2508142110314508a3d071f89b321023': created (progress 0%)
0:00:33 Job 'j-2508142110314508a3d071f89b321023': created (progress 0%)
0:00:43 Job 'j-2508142110314508a3d071f89b321023': running (progress N/A)
0:00:55 Job 'j-2508142110314508a3d071f89b321023': running (progress N/A)
0:01:11 Job 'j-2508142110314508a3d071f89b321023': running (progress N/A)
0:01:30 Job 'j-2508142110314508a3d071f89b321023': running (progress N/A)
0:01:54 Job 'j-2508142110314508a3d071f89b321023': finished (progress 100%)
✅ Descargado Atitlan_RAW 2025-02-10
0:00:00 Job 'j-25081421124940ddad7f05a36cb31897': send 'start'
0:00:13 Job 'j-25081421124940ddad7f05a36cb31897': created (progress 0%)
0:00:18 Job 'j-25081421124940ddad7f05a36cb31897': created (progress 0%)
0:0

: 

: 

In [5]:
# === Paso 4 (Amatitlán solamente): un job por fecha, con retry y cálculo local de índices ===
import os, time, glob, datetime as dt
import numpy as np
import rasterio

# Fechas para AMATITLÁN (2025) según tu lista
fechas_amatitlan = sorted(set([
    # Febrero
    "2025-02-07", "2025-02-12", "2025-02-22", "2025-02-27",
    # Marzo
    "2025-03-04", "2025-03-09", "2025-03-14", "2025-03-19", "2025-03-24", "2025-03-26",
    # Abril
    "2025-04-03", "2025-04-13", "2025-04-15", "2025-04-18", "2025-04-28",
    # Mayo
    "2025-05-13",
    # Junio
    "2025-06-14",
    # Julio
    "2025-07-17", "2025-07-22", "2025-07-24", "2025-07-27",
]))

def _next_day(d):  # 'YYYY-MM-DD' -> 'YYYY-MM-DD' + 1
    return (dt.datetime.fromisoformat(d) + dt.timedelta(days=1)).date().isoformat()

def _write_singleband_tif(ref_src, array, out_path, nodata=np.nan):
    profile = ref_src.profile.copy()
    profile.update(count=1, dtype="float32", nodata=nodata)
    with rasterio.open(out_path, "w", **profile) as dst:
        dst.write(array.astype("float32"), 1)

def _calc_indices_and_save(src_path, out_base, date_str):
    with rasterio.open(src_path) as src:
        if src.count < 5:
            print(f"Saltando {src_path}: se esperaban 5 bandas (B03,B04,B05,B08,SCL).")
            return
        B03 = src.read(1).astype("float32")
        B04 = src.read(2).astype("float32")
        B05 = src.read(3).astype("float32")
        B08 = src.read(4).astype("float32")
        SCL = src.read(5)

        # Máscara: agua y sin nubes/sombras
        is_water = (SCL == 6)
        is_cloud = ((SCL == 3) | (SCL == 8) | (SCL == 9) | (SCL == 10) | (SCL == 11))
        mask = is_water & (~is_cloud)

        # Escala a reflectancia
        scale = 0.0001
        B03s, B04s, B05s, B08s = B03*scale, B04*scale, B05*scale, B08*scale
        eps = 1e-6

        ndci = (B05s - B04s) / (B05s + B04s + eps)
        ndvi = (B08s - B04s) / (B08s + B04s + eps)
        ndwi = (B03s - B08s) / (B03s + B08s + eps)

        ndci[~mask] = np.nan
        ndvi[~mask] = np.nan
        ndwi[~mask] = np.nan

        for name, arr in [("ndci", ndci), ("ndvi", ndvi), ("ndwi", ndwi)]:
            out_dir = os.path.join(out_base, name)
            os.makedirs(out_dir, exist_ok=True)
            _write_singleband_tif(src, arr, os.path.join(out_dir, f"{name}_{date_str}.tif"))

def _download_one_day(con, bbox, collection_id, date_str, raw_dir, job_title, max_retries=4):
    start, end = date_str, _next_day(date_str)
    cube = con.load_collection(
        collection_id,
        spatial_extent=bbox,
        temporal_extent=[start, end],
        bands=["B03","B04","B05","B08","SCL"],
    )
    job = cube.save_result(format="GTIFF")
    backoff = 2.0
    for attempt in range(1, max_retries+1):
        try:
            batch = con.create_job(job, title=f"{job_title}_{date_str}")
            res = batch.start_and_wait()
            os.makedirs(raw_dir, exist_ok=True)
            res.download_results(raw_dir)
            print(f"✅ Descargado {job_title} {date_str}")
            return True
        except Exception as e:
            msg = str(e)
            if "Too Many Requests" in msg or "429" in msg:
                print(f"⚠️ 429 en {date_str}. Reintentando en {backoff:.1f}s (intento {attempt}/{max_retries})...")
                time.sleep(backoff)
                backoff *= 1.8
            else:
                print(f"❌ Falló {date_str}: {e}")
                return False
    print(f"❌ Agotados reintentos para {date_str}")
    return False

def pipeline_per_area(con, bbox, area_name, out_base, fechas):
    raw_dir = os.path.join(out_base, "raw")
    os.makedirs(raw_dir, exist_ok=True)
    ok = 0
    for d in fechas:
        if _download_one_day(con, bbox, collection, d, raw_dir, f"{area_name}_RAW"):
            # localizar el tif recién bajado (multibanda)
            tifs = sorted(glob.glob(os.path.join(raw_dir, "**", "*.tif"), recursive=True), key=os.path.getmtime)
            if not tifs:
                print(f"⚠️ No se encontró el GeoTIFF para {d}")
                continue
            latest = tifs[-1]
            _calc_indices_and_save(latest, out_base, d)
            ok += 1
    print(f"✔️ {area_name}: {ok}/{len(fechas)} fechas procesadas")

# Ejecutar SOLO Amatitlán (asegúrate de tener 'amatitlan_bbox' y 'collection' definidos arriba)
# collection = "SENTINEL2_L2A"  # por si lo necesitas
pipeline_per_area(con, amatitlan_bbox, "Amatitlan", "out_amatitlan", fechas_amatitlan)


0:00:00 Job 'j-2508142327214bd2b2dd76998fbe3a30': send 'start'
0:00:13 Job 'j-2508142327214bd2b2dd76998fbe3a30': created (progress 0%)
0:00:18 Job 'j-2508142327214bd2b2dd76998fbe3a30': created (progress 0%)
0:00:25 Job 'j-2508142327214bd2b2dd76998fbe3a30': created (progress 0%)
0:00:33 Job 'j-2508142327214bd2b2dd76998fbe3a30': created (progress 0%)
0:00:43 Job 'j-2508142327214bd2b2dd76998fbe3a30': created (progress 0%)
0:00:56 Job 'j-2508142327214bd2b2dd76998fbe3a30': running (progress N/A)
0:01:11 Job 'j-2508142327214bd2b2dd76998fbe3a30': running (progress N/A)
0:01:31 Job 'j-2508142327214bd2b2dd76998fbe3a30': running (progress N/A)
0:01:55 Job 'j-2508142327214bd2b2dd76998fbe3a30': finished (progress 100%)


C:\Users\usuario\AppData\Local\Temp\ipykernel_12948\2489755301.py:80: UserDeprecationWarning: Call to deprecated method download_results. (Instead use `BatchJob.get_results` and the more flexible download functionality of `JobResults`) -- Deprecated since version 0.4.10.
  res.download_results(raw_dir)
C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openeo\rest\job.py:199: UserDeprecationWarning: Call to deprecated method get_result. (Use `BatchJob.get_results` instead.) -- Deprecated since version 0.4.10.
  return self.get_result().download_files(target)
C:\Users\usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openeo\rest\job.py:203: UserDeprecationWarning: Call to deprecated class _Result. (Use `JobResults` instead) -- Deprecated since version 0.4.10.
  return _Result(self)


✅ Descargado Amatitlan_RAW 2025-02-07
0:00:00 Job 'j-250814232936493bb4df68d9af7a54d1': send 'start'
0:00:13 Job 'j-250814232936493bb4df68d9af7a54d1': created (progress 0%)
0:00:18 Job 'j-250814232936493bb4df68d9af7a54d1': created (progress 0%)
0:00:25 Job 'j-250814232936493bb4df68d9af7a54d1': created (progress 0%)
0:00:33 Job 'j-250814232936493bb4df68d9af7a54d1': running (progress N/A)
0:00:43 Job 'j-250814232936493bb4df68d9af7a54d1': running (progress N/A)
0:00:55 Job 'j-250814232936493bb4df68d9af7a54d1': running (progress N/A)
0:01:11 Job 'j-250814232936493bb4df68d9af7a54d1': running (progress N/A)
0:01:30 Job 'j-250814232936493bb4df68d9af7a54d1': running (progress N/A)
0:01:54 Job 'j-250814232936493bb4df68d9af7a54d1': running (progress N/A)
0:02:24 Job 'j-250814232936493bb4df68d9af7a54d1': running (progress N/A)
0:03:02 Job 'j-250814232936493bb4df68d9af7a54d1': finished (progress 100%)
✅ Descargado Amatitlan_RAW 2025-02-12
0:00:00 Job 'j-250814233259460c94864e47bb92b47d': send 'sta

## 5) Conversión a NumPy y series de tiempo por lago


In [ ]:
import os, glob
import numpy as np
import pandas as pd
import rasterio

def _extract_date_from_name(path):
    toks = os.path.basename(path).replace("_","-").split("-")
    for tok in toks:
        if len(tok)==10 and tok[4]=="-" and tok[7]=="-":
            return tok
    return None

def _nanify_nodata(arr, nodata):
    if nodata is None:
        return arr
    # Si nodata es un número finito, conviértelo a NaN
    if not np.isnan(nodata):
        arr = arr.copy()
        arr[arr == nodata] = np.nan
    return arr

def mean_time_series_from_subfolders(base_folder):
    """
    Espera estructura:
      base_folder/
        ndci/ ndci_YYYY-MM-DD.tif (monobanda)
        ndvi/ ndvi_YYYY-MM-DD.tif (monobanda)
        ndwi/ ndwi_YYYY-MM-DD.tif (monobanda)
    Devuelve un DataFrame con columnas: date, ndci_mean, ndvi_mean, ndwi_mean
    """
    buckets = {"ndci": {}, "ndvi": {}, "ndwi": {}}

    for idx in ["ndci", "ndvi", "ndwi"]:
        for path in sorted(glob.glob(os.path.join(base_folder, idx, "*.tif"))):
            date_str = _extract_date_from_name(path)
            if date_str is None:
                continue
            with rasterio.open(path) as src:
                a = src.read(1).astype("float32")
                a = _nanify_nodata(a, src.nodata)
                m = float(np.nanmean(a)) if np.isfinite(a).any() else np.nan
            buckets[idx][date_str] = m

    # Unir dicts por fecha
    all_dates = sorted(set().union(*[set(d.keys()) for d in buckets.values()]))
    rows = []
    for d in all_dates:
        rows.append({
            "date": pd.to_datetime(d),
            "ndci_mean": buckets["ndci"].get(d, np.nan),
            "ndvi_mean": buckets["ndvi"].get(d, np.nan),
            "ndwi_mean": buckets["ndwi"].get(d, np.nan),
        })
    return pd.DataFrame(rows).sort_values("date").reset_index(drop=True)

# Uso:
ts_atitlan   = mean_time_series_from_subfolders("out_atitlan")
ts_amatitlan = mean_time_series_from_subfolders("out_amatitlan")
display(ts_atitlan.head(), ts_amatitlan.head())


## 6) Análisis temporal: visualización y detección de picos

In [ ]:
def plot_ndci_timeseries(ts_atitlan, ts_amatitlan):
    plt.figure()
    if len(ts_atitlan)>0:
        plt.plot(ts_atitlan["date"], ts_atitlan["ndci_mean"], label="Atitlán — NDCI")
    if len(ts_amatitlan)>0:
        plt.plot(ts_amatitlan["date"], ts_amatitlan["ndci_mean"], label="Amatitlán — NDCI")
    plt.xlabel("Fecha"); plt.ylabel("NDCI promedio"); plt.title("Evolución temporal de NDCI"); plt.legend(); plt.show()

def detect_peaks(ts_df, thr=0.10):
    return ts_df[ts_df["ndci_mean"] > thr].copy()

# Ejemplo de uso (descomenta cuando existan ts_*):
# plot_ndci_timeseries(ts_atitlan, ts_amatitlan)
# peaks_atitlan   = detect_peaks(ts_atitlan, thr=0.10)
# peaks_amatitlan = detect_peaks(ts_amatitlan, thr=0.10)
# display(peaks_atitlan.head(), peaks_amatitlan.head())

## 7) Análisis espacial: mapas de NDCI
Muestra mapas raster sencillos (matplotlib). Para mapas interactivos, puedes usar **folium** y `ImageOverlay`.

In [ ]:
def show_map_example(path_tif, title="Mapa NDCI"):
    with rasterio.open(path_tif) as src:
        arr = src.read(1)  # NDCI
        plt.figure()
        plt.imshow(arr)
        plt.colorbar(label="NDCI")
        plt.title(title)
        plt.axis("off")
        plt.show()

# Ejemplo (ajusta la ruta a un archivo existente):
# import os
# tifs = sorted(glob.glob("out_atitlan/**/*.tif", recursive=True))
# if tifs:
#     show_map_example(tifs[0], title=f"NDCI Atitlán — {os.path.basename(tifs[0])}")

## 8) Correlación NDCI con NDVI/NDWI

In [ ]:
def corr_table(ts_df):
    return ts_df[["ndci_mean","ndvi_mean","ndwi_mean"]].corr()

# Ejemplo (una vez tengas ts_*):
# corr_atitlan   = corr_table(ts_atitlan)
# corr_amatitlan = corr_table(ts_amatitlan)
# display(corr_atitlan, corr_amatitlan)

## 9) Modelos
### 9.1 Serie temporal (ARIMA básico sobre `ndci_mean`)
Ajusta órdenes (p,d,q) tras revisar ACF/PACF.

In [ ]:
# Modelo ARIMA simple
# !pip install statsmodels
import statsmodels.api as sm

def arima_forecast(ts_df, steps=4):
    y = ts_df.set_index("date")["ndci_mean"].asfreq("7D").interpolate()
    model = sm.tsa.ARIMA(y, order=(1,0,1))
    res = model.fit()
    fc = res.get_forecast(steps=steps)
    print(res.summary())
    return y, fc

# Ejemplo (cuando exista ts_*):
# y_at, fc_at = arima_forecast(ts_atitlan, steps=4)
# y_am, fc_am = arima_forecast(ts_amatitlan, steps=4)

### 9.2 Clasificación (¿hay cianobacteria en un píxel? 1/0)
Usa NDVI/NDWI como *features* y un umbral de NDCI para etiquetas.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def build_xy_from_tif(path_tif, ndci_thr=0.10):
    with rasterio.open(path_tif) as src:
        ndci = src.read(1); ndvi = src.read(2); ndwi = src.read(3)
    mask = ~np.isnan(ndci) & ~np.isnan(ndvi) & ~np.isnan(ndwi)
    X = np.column_stack([ndvi[mask], ndwi[mask]])
    y = (ndci[mask] > ndci_thr).astype(int)
    return X, y

def train_logreg_on_tif(path_tif, ndci_thr=0.10):
    X, y = build_xy_from_tif(path_tif, ndci_thr=ndci_thr)
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=42)
    clf = LogisticRegression(max_iter=1000).fit(Xtr, ytr)
    print(classification_report(yte, clf.predict(Xte)))
    return clf

# Ejemplo (ajusta la ruta):
# sample_tif = sorted(glob.glob("out_atitlan/**/*.tif", recursive=True))[0]
# clf = train_logreg_on_tif(sample_tif, ndci_thr=0.10)

### 9.3 Modelo híbrido
Primero pronostica `ndci_mean` (ARIMA) y úsalo como *feature* adicional (junto a NDVI/NDWI y otras variables) para clasificar si un área estará contaminada.

In [ ]:
# Esqueleto de híbrido (ajústalo a tu pipeline de datos)
def hybrid_features_from_tif(path_tif, ndci_forecast_scalar, ndci_thr=0.10):
    with rasterio.open(path_tif) as src:
        ndci = src.read(1); ndvi = src.read(2); ndwi = src.read(3)
    mask = ~np.isnan(ndci) & ~np.isnan(ndvi) & ~np.isnan(ndwi)
    X = np.column_stack([ndvi[mask], ndwi[mask], np.full(mask.sum(), ndci_forecast_scalar, dtype=float)])
    y = (ndci[mask] > ndci_thr).astype(int)
    return X, y

# Ejemplo de entrenamiento cuando tengas un pronóstico escalar del lago (p.ej. media pronosticada):
# ndci_fc_scalar = float(fc_at.predicted_mean.mean())
# Xh, yh = hybrid_features_from_tif(sample_tif, ndci_fc_scalar, ndci_thr=0.10)
# Xtr, Xte, ytr, yte = train_test_split(Xh, yh, test_size=0.3, random_state=42)
# clf_h = LogisticRegression(max_iter=1000).fit(Xtr, ytr)
# print(classification_report(yte, clf_h.predict(Xte)))

## 10) Visualización de proyecciones en mapa
Aplica el clasificador píxel a píxel para construir un raster binario (0/1) y muéstralo.

In [ ]:
def project_map_from_classifier(path_tif, clf, ndci_forecast_scalar=None):
    with rasterio.open(path_tif) as src:
        profile = src.profile
        ndci = src.read(1); ndvi = src.read(2); ndwi = src.read(3)
    mask = ~np.isnan(ndvi) & ~np.isnan(ndwi)
    if ndci_forecast_scalar is None:
        X = np.column_stack([ndvi[mask], ndwi[mask]])
    else:
        X = np.column_stack([ndvi[mask], ndwi[mask], np.full(mask.sum(), ndci_forecast_scalar, dtype=float)])
    yhat = np.zeros_like(ndvi, dtype=float)
    yhat[mask] = clf.predict(X)
    plt.figure()
    plt.imshow(yhat)
    plt.title("Proyección binaria (1=probable cianobacteria)")
    plt.axis("off")
    plt.show()
    return yhat

# Ejemplo (ajusta rutas y modelos antes de ejecutar):
# yhat_map = project_map_from_classifier(sample_tif, clf)

## Notas finales
- Ajusta el **umbral NDCI** según literatura/valores observados.
- Para robustez, promedia múltiples fechas cercanas antes de clasificar.
- Incorpora variables externas (p. ej., temperatura/precipitación de Weatherspark) para el modelo híbrido.
- Versiona tu trabajo y documenta decisiones de preprocesamiento y modelos.
